In [1]:

from tensorflow.keras.applications import VGG16  # Örnek olarak VGG16 modelini kullanalım
from tensorflow.keras.models import Model
import os
import cv2
import numpy as np
!pip install numpy opencv-python scikit-learn tensorflow

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


In [2]:
from google.colab import drive

# Google Drive'a bağlanma
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Veri setinin bulunduğu klasörün yolu
data_folder = '/content/drive/My Drive/lung_image_sets'


In [4]:
DATADIR = '/content/drive/My Drive/lung_image_sets'

CATEGORIES = ["lung_adenocarcinoma","lung_benign tissue","lung_squamous cell carcinoma"]
classes = os.listdir(DATADIR)

In [45]:
IMG_SIZE = 50

In [50]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                pass
create_all_data()

In [51]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  15000


In [52]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [53]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [54]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [55]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(10800, 50, 50, 3)
(3000, 50, 50, 3)
(1200, 50, 50, 3)
(10800,)
(3000,)
(1200,)


In [56]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=3)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=3)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=3)

In [57]:
from tensorflow.keras.applications import ResNet50

# Önceden eğitilmiş modeli yükleme (ResNet50 kullanımı)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(50, 50, 3))

In [59]:
# Transfer Learning modeli oluşturma
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [60]:
# Modeli derleme
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [61]:

model.fit(x_train, train_yCl, epochs=5, batch_size=32, validation_data=(x_val, valid_yCl))

Epoch 1/5
338/338 [==============================] - 1591s 5s/step - loss: 0.4809 - accuracy: 0.8776 - val_loss: 3.8798 - val_accuracy: 0.6400
Epoch 2/5
338/338 [==============================] - 1561s 5s/step - loss: 0.2041 - accuracy: 0.9246 - val_loss: 0.5461 - val_accuracy: 0.8317
Epoch 3/5
338/338 [==============================] - 1569s 5s/step - loss: 0.1418 - accuracy: 0.9458 - val_loss: 0.2429 - val_accuracy: 0.9100
Epoch 4/5
338/338 [==============================] - 1556s 5s/step - loss: 0.1095 - accuracy: 0.9584 - val_loss: 0.2470 - val_accuracy: 0.9050
Epoch 5/5
338/338 [==============================] - 1539s 5s/step - loss: 0.0875 - accuracy: 0.9676 - val_loss: 0.1830 - val_accuracy: 0.9175


In [62]:
# Test verisiyle doğruluk oranını hesaplama
test_accuracy = model.evaluate(x_test, y_test)[1]
print("Test Accuracy:", test_accuracy)

# Test verisi üzerinde tahmin yapma
y_pred = model.predict(x_test)

# Performans metriklerini hesaplama
precision = precision_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='weighted')
recall = recall_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='weighted')
f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Karmaşıklık matrisi
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
print("Confusion Matrix:\n", conf_matrix)

# Sınıflandırma raporu
class_report = classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
print("Classification Report:\n", class_report)

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2066, in test_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2049, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2037, in run_step  **
        outputs = model.test_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1919, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/usr